In [10]:
import os, re
import pandas as pd 
from tqdm import notebook as tqdm 
folder = "../.."

import os 
import pandas as pd 
import json
with open("../assets/Bibles/ESV.json") as file: 
    ESV = json.load(file)
    
bible = {}

b_versions = ['AKJV','Geneva', 'ODRV','Douay-Rheims', 'Tyndale', 'Wycliffe','Vulgate']
for bname in b_versions:
    data = pd.read_csv(f"{folder}/Early-Modern-Sermons/assets/Bibles/{bname}.csv",header=None)
    data = data.to_dict(orient="records")
    for entry in tqdm.tqdm(data):
        key = entry[0]
        v_id = key.split(" (")[0]
        text = entry[6]
        if re.search("Douay-Rheims",key):
            if re.sub("Douay-Rheims","ODRV",key) in bible: continue
        if len(text.split(" ")) < 200:
            bible[key] = f"{v_id} {text}"

        parts = re.split(r'(?<=[\.\?]) (?=[A-Z])|(?<=[\!\:\;])', text)
        parts = [re.sub(r'\s+', ' ', p).strip() for p in parts if len(p.strip(" ")) > 0]
        if (len(parts[0].split(" ")) <= 5 or len(parts[-1].split(" ")) <= 5 or re.search(r"\&\w+\;",parts[0])): 
            for pidx, p in enumerate(parts): continue
        elif len(parts) > 1:  
            for pidx, p in enumerate(parts):
              p_id = f"{key} - {pidx}"
              if len(p) == 0: continue
              if re.search(r"\&\w+\;",p) or len(p.split(" ")) <= 5: continue
              bible[p_id] = f"Part {pidx+1} of {v_id}: {p}"
bible_verses = list(bible.values())
bible_ids = list(bible.keys())
id_to_idx = {v_id:idx for idx, v_id in enumerate(bible_ids)}
len(bible_verses)

  0%|          | 0/14737 [00:00<?, ?it/s]

  0%|          | 0/35811 [00:00<?, ?it/s]

  0%|          | 0/7954 [00:00<?, ?it/s]

  0%|          | 0/9622 [00:00<?, ?it/s]

  0%|          | 0/35809 [00:00<?, ?it/s]

276627

In [16]:
search = '2 Corinthians 2.8'
if search in ESV: 
    print(ESV[search])
for v_id in bible:
    if re.search(rf"^{search}",v_id):
        if " - " in v_id: continue
        # if 'Wycliffe' not in v_id: continue 
        # if "AKJV" not in v_id and "Geneva" not in v_id: continue
        # if "ODRV" not in v_id and "Vulgate" not in v_id and "Douay-Rheims" not in v_id: continue
        # if " - 0" not in v_id: continue
        # print(v_id)
        print(f'{v_id}\t{bible[v_id]}')

so you should rather turn to forgive and comfort him, or he may be overwhelmed by excessive sorrow.
2 Corinthians 2.8 (AKJV)	2 Corinthians 2.8 Wherefore I beseech you, that you would confirme your loue towards him.
2 Corinthians 2.8 (Geneva)	2 Corinthians 2.8 Wherefore, I pray you, that you woulde confirme your loue towards him.
2 Corinthians 2.8 (ODRV)	2 Corinthians 2.8 For the which cause I beseech you that you confirme charitie toward him.
2 Corinthians 2.8 (Tyndale)	2 Corinthians 2.8 Wherfore I exhorte you that love maye have stregth over him.
2 Corinthians 2.8 (Vulgate)	2 Corinthians 2.8 Propter quod obsecro vos, ut confirmetis in illum caritatem.


In [ ]:
fname = "parallel_verses_UNCONFIDENT - LABELED"
parallel = pd.read_csv(f"{folder}/Early-Modern-Sermons/assets/QP_Datasets/{fname}.csv").to_dict(orient='records')
import json 

In [ ]:
output = {}
found = {}
save = []

for item in parallel: 
  key = (item['ESV'],item['version'])
  if item['toCheck'] is False: 
    save.append(item)
    continue 
  key2 =  (item['ESV'],item['text'])
  if key2 not in output: output[key2] = {}
  output[key2][(item['Parallel'], item['version'])] = (item['prediction'], item['verse_text'])

save = pd.DataFrame(save)
save.to_csv(f"{folder}/Early-Modern-Sermons/assets/QP_Datasets/parallel_verses_CONFIDENT - DSV3.csv",index=False)
len(save)

In [18]:
labeled_dict = {}
with open(f"{folder}/QP/PT_verified.txt",'r') as file: 
    labels = file.read()
labels = labels.split("--------------------------")
for item in labels:
    if len(item) <= 1: continue 
    item = item.split("\n")
    v_id = item[1].split("\t")[0]
    labeled_dict[v_id] = []
    for entry in item[2:]: 
        if entry == '': continue 
        labeled_dict[v_id].append(entry.split("\t")[0])
with open(f"{folder}/Early-Modern-Sermons/assets/QP_Datasets/confident_labeled.json",'w+') as file: 
    json.dump(list(labeled_dict.values()), file)
len(labeled_dict)

354

In [ ]:
def fix_name(v_id):
  if "1 Kings" in v_id: v_id = "3 Kings" + v_id.split("1 Kings")[-1]
  elif "2 Kings" in v_id: v_id = "4 Kings" + v_id.split("2 Kings")[-1]
  elif "1 Samuel" in v_id: v_id = "1 Kings" + v_id.split("1 Samuel")[-1]
  elif "2 Samuel" in v_id: v_id = "2 Kings" + v_id.split("2 Samuel")[-1]
  elif re.search(r"^\d+ Chronicles",v_id): v_id = re.sub(r"Chronicles","Paralipomenon",v_id)
  return v_id
with open(f"{folder}/QP/PT_to_verify.txt",'w+') as file: 
    for ESV_id, pt_list in output.items(): 
        if "Psalms" not in ESV_id[0]: continue 
        file.write(f"{ESV_id[0]}\t{ESV_id[1]}\n")
        seen = {}
        for pt, pt_text in pt_list.items(): 
            v_id = f"{pt[0]} ({pt[1]})"
            seen[v_id] = None 
            file.write(f"{pt_text[0]}\t{v_id}\t{pt_text[1]}\n")
        for b in b_versions: 
            v_id = f"{ESV_id[0]} ({b})"
            if b in ['Douay-Rheims','ODRV','Vulgate']: 
                v_id = fix_name(v_id)
            if v_id in bible and v_id not in seen: 
                seen[v_id] = None 
                file.write(f"{v_id}\t{bible[v_id]}\n")
            
            if b in ['ODRV','Vulgate','Wycliffe']:
                v_id = v_id.split(" (")[0]
                bc = v_id.split(".")[0]
                v = v_id.split(".")[-1]
                v = int(v)
                for i in range(0,2): 
                    v_id = f"{bc}.{v+i} ({b})"
                    if v_id in bible: 
                        seen[v_id] = None 
                        file.write(f"{v_id}\t{bible[v_id]}\n")
                c = re.findall('\d+',bc)[-1]
                bc = re.sub(c,f"{int(c)-1}",bc)
                
                for i in range(-1,3): 
                    v_id = f"{bc}.{v+i} ({b})"
                    if v_id in bible: 
                        seen[v_id] = None 
                        file.write(f"{v_id}\t{bible[v_id]}\n")
        file.write('--------------------------\n')
        

In [ ]:
for i in range(1,26): 
    print(f'Job 41.{i+9}','\t',f'Job 41.{i+9}')
    print(f'Job 41.{i+9} (AKJV)','\t',bible[f'Job 41.{i+9} (AKJV)'])
    for b in ['Douay-Rheims','Vulgate','Geneva']: 
        print(f"Job 41.{i} ({b})",'\t',bible[f"Job 41.{i} ({b})"])
    print()

In [ ]:
to_verify = ['Esther 8.10 (Douay-Rheims)', 'Job 41.2 (AKJV)', 'Job 41.3 (AKJV)', 'Job 41.27 (AKJV)', 'Psalms 10.3 (Geneva)', 'Psalms 13.6 (AKJV)', 'Psalms 15.2 (Vulgate)', 'Psalms 49.18 (AKJV)', 'Psalms 49.19 (AKJV)', 'Psalms 60.4 (Geneva)', 'Psalms 68.14 (Geneva)', 'Canticles 1.1 (AKJV)', 'Canticles 6.1 (AKJV)', 'Jeremiah 46.17 (AKJV)', 'Jeremiah 48.9 (AKJV)', 'Ezekiel 21.13 (Geneva)', 'Ezekiel 42.18 (Geneva)', 'Daniel 4.1 (AKJV)', 'Daniel 4.36 (AKJV)', 'Ecclesiastes 5.8 (Geneva)']

for item in to_verify: 
    search = item.split("(")[0]
    for v_id in bible:
        if re.search(rf"^{search}",v_id):
            if " - " in v_id: continue
            # if 'Wycliffe' not in v_id: continue 
            # if "AKJV" not in v_id and "Geneva" not in v_id: continue
            # if "ODRV" not in v_id and "Vulgate" not in v_id and "Douay-Rheims" not in v_id: continue
            # if " - 0" not in v_id: continue
            # print(v_id)
            print(f'{v_id}\t{bible[v_id]}')
    print('--------------------------')

In [ ]:
search = "Psalms 7\.9 "
for v_id in bible:
    if re.search(rf"^{search}",v_id):
        if " - " in v_id: continue
        # if 'Wycliffe' not in v_id: continue 
        # if "AKJV" not in v_id and "Geneva" not in v_id: continue
        if "ODRV" not in v_id and "Vulgate" not in v_id and "Douay-Rheims" not in v_id: continue
        # if " - 0" not in v_id: continue
        # print(v_id)
        print(f'{v_id}\t{bible[v_id]}')